In [1]:
from fastapi import FastAPI
from pytrends.request import TrendReq
import pandas as pd
import time

app = FastAPI()

def obtener_datos_google_trends_semanal(topic):
    try:
        # Intentar obtener datos de Google Trends
        pytrends = TrendReq(hl='es', tz=360)
        pytrends.build_payload(kw_list=[topic], timeframe='2014-01-01 2024-03-31', geo='US')
        data = pytrends.interest_over_time()
        
        # Convertir datos a frecuencia semanal
        data_semanal = data.resample('W').mean()
        
        return data_semanal
    except Exception as e:
        # Si ocurre un error, imprimirlo y esperar antes de volver a intentar
        print(f"Error al obtener datos de Google Trends: {e}")
        time.sleep(60)  # Esperar 60 segundos 
        return obtener_datos_google_trends_semanal(topic)

# Endpoint para Google Trends
@app.get("/google-trends/{topic}") # donde meter los topics
def google_trends_handler(topic: str):
    trends_data = obtener_datos_google_trends_semanal(topic) # Obtenemos los datos de Google Trends para el tema proporcionado
    return trends_data.to_dict() # Devolver los datos de tendencias como respuesta
